In [42]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import functional as F
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torchvision import transforms, datasets
from torch.autograd import Variable

In [43]:
# General steps for using PyTorch

# Step 1. Load Dataset
# Step 3. Create Model Class
# Step 4. Instantiate Model Class
# Step 5. Instantiate Loss Class
# Step 6. Instantiate Optimizer Class
# Step 7. Train Model

In [44]:
# Getting the data

X = pd.read_csv("nba.games.stats.csv")
print(X.shape)

# trainX = X[:6888]
# testX = X[6888:]

trainX = X[:6593]
testX = X[6593:]

# trainX = X[:1000]
# testX = X[1000:1200]

X.head(10)

(9840, 41)


,Unnamed: 0,Team,Game,Date,Home,Opponent,WINorLOSS,TeamPoints,OpponentPoints,FieldGoals,...,Opp.FreeThrows,Opp.FreeThrowsAttempted,Opp.FreeThrows.,Opp.OffRebounds,Opp.TotalRebounds,Opp.Assists,Opp.Steals,Opp.Blocks,Opp.Turnovers,Opp.TotalFouls
0,1,ATL,1,2014-10-29,Away,TOR,L,102,109,40,...,27,33,0.818,16,48,26,13,9,9,22
1,2,ATL,2,2014-11-01,Home,IND,W,102,92,35,...,18,21,0.857,11,44,25,5,5,18,26
2,3,ATL,3,2014-11-05,Away,SAS,L,92,94,38,...,27,38,0.711,11,50,25,7,9,19,15
3,4,ATL,4,2014-11-07,Away,CHO,L,119,122,43,...,20,27,0.741,11,51,31,6,7,19,30
4,5,ATL,5,2014-11-08,Home,NYK,W,103,96,33,...,8,11,0.727,13,44,26,2,6,15,29
5,6,ATL,6,2014-11-10,Away,NYK,W,91,85,27,...,7,12,0.583,11,40,23,4,2,15,26
6,7,ATL,7,2014-11-12,Home,UTA,W,100,97,39,...,6,12,0.500,8,30,28,12,8,11,17
7,8,ATL,8,2014-11-14,Home,MIA,W,114,103,42,...,23,25,0.920,5,32,27,10,3,14,20
8,9,ATL,9,2014-11-15,Away,CLE,L,94,127,40,...,10,13,0.769,12,47,39,12,2,13,14
9,10,ATL,10,2014-11-18,Home,LAL,L,109,114,41,...,14,22,0.636,13,44,24,7,0,11,24


In [45]:
# Getting the target values

target = X.pop('WINorLOSS').values

train_target = trainX.pop('WINorLOSS').values
test_target = testX.pop('WINorLOSS').values

In [46]:
# Converting target feature (WIN, LOSS) to binary values (1, 0)

binaryTarget = [0 if x is 'L' else 1 for x in target]
train_binaryTarget = [0 if x is 'L' else 1 for x in train_target]
test_binaryTarget = [0 if x is 'L' else 1 for x in test_target]

binaryTarget_tensor = torch.tensor(binaryTarget)
train_binaryTarget_tensor = torch.tensor(train_binaryTarget)
test_binaryTarget_tensor = torch.tensor(test_binaryTarget)

In [47]:
# Feature Selection

# 17 features
singleTeamFeatureSet = ['TeamPoints', 'FieldGoals', 'FieldGoalsAttempted', 'FieldGoals.', 'X3PointShots', 
                            'X3PointShotsAttempted', 'X3PointShots.', 'FreeThrows', 'FreeThrowsAttempted',
                                'FreeThrows.', 'OffRebounds', 'TotalRebounds', 'Assists', 'Steals', 'Blocks',
                                    'Turnovers', 'TotalFouls']

# 13 features
singleTeamOffensiveFeatureSet = ['TeamPoints', 'FieldGoals', 'FieldGoalsAttempted', 'FieldGoals.', 'X3PointShots',
                                     'X3PointShotsAttempted', 'X3PointShots.', 'FreeThrows', 'FreeThrowsAttempted',
                                        'FreeThrows.', 'OffRebounds', 'Assists', 'Turnovers']

# 16 features 
singleTeamDefensiveFeatureSet = ['OpponentPoints', 'Steals', 'Blocks', 'TotalFouls', 'Opp.FieldGoals',
                                     'Opp.FieldGoalsAttempted', 'Opp.FieldGoals.', 'Opp.3PointShots',
                                         'Opp.3PointShotsAttempted', 'Opp.3PointShots.', 'Opp.FreeThrows',
                                             'Opp.FreeThrowsAttempted', 'Opp.FreeThrows.', 'Opp.OffRebounds',
                                                 'Opp.Assists', 'Opp.Turnovers']

In [48]:
# Trim data to contain only select features

train_featuresX = trainX[singleTeamDefensiveFeatureSet].copy()
test_featuresX = testX[singleTeamDefensiveFeatureSet].copy()

In [49]:
# Converting data to tensor

X_tensor_train = torch.tensor(train_featuresX.values)
X_tensor_test = torch.tensor(test_featuresX.values)

print(X_tensor_train.shape)
print(X_tensor_test.shape)

torch.Size([6593, 16])
torch.Size([3247, 16])


In [50]:
# Define the logistic regression model

class LogisticRegressionModel(nn.Module):
    def __init__(self):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(16, 1)
        
    def forward(self, x):
        pred = torch.sigmoid(self.linear(x))
        return pred

In [51]:
# Create an instance of the model

lr_model = LogisticRegressionModel()

In [52]:
# Define the loss function and the optimizer

# criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss() # Binary Cross Entropy
optimizer = torch.optim.SGD(lr_model.parameters(), lr=0.001) # Stochastic Gradient Descent

In [53]:
# Training the model

for epoch in range(10):
    
    # Zero gradients at the start of each epoch
    optimizer.zero_grad()
    
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = lr_model(X_tensor_train.float())
    
    # Converts the tensor object from multidimensional to a single dimension 
    y_pred_squeezed = y_pred.squeeze()
    
    # Compute and print loss
    loss = criterion(y_pred_squeezed, train_binaryTarget_tensor.float())
    print(epoch, loss.data)
    
    # Perform a backward pass, and update the weights
    loss.backward()
    optimizer.step()

0 tensor(2.1458)
1 tensor(1.6541)
2 tensor(1.7876)
3 tensor(1.6621)
4 tensor(1.7912)
5 tensor(1.6587)
6 tensor(1.7891)
7 tensor(1.6545)
8 tensor(1.7864)
9 tensor(1.6501)


In [54]:
# Testing the Model

correct = 0
total = len(test_binaryTarget_tensor)

with torch.no_grad():
    for game, target in zip(X_tensor_test, test_binaryTarget_tensor):
        
        y_pred = lr_model(game.float())

        if y_pred[0] < 0.5:
            temp = torch.tensor(0)
        else:
            temp = torch.tensor(1)

        if temp == target:
            correct += 1

print('The accuracy is: ' + str(100*(correct/total)) + '%')
    

The accuracy is: 52.510009239297815%
